In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

: 

In [ ]:
def f(x):
    return 3*x**2 - 4*x +5

: 

In [ ]:
f(3.0)

: 

In [ ]:
xs = np.arange(-10, 10, 0.1)
ys = f(xs)
plt.plot(xs, ys)

: 

In [ ]:
h = 0.000000000000001
x = 2/3
(f(x+h) - f(x))/h

: 

Derivative of F(x) = ax + b

In [ ]:
a = 2
x = 1
b = 0
h = 0.000000001
(f_1(a,x+h,b) - f_1(a,x,b))/h

: 

In [ ]:
# We need to create tissue to connect the neurons together
class Value:
    def __init__(self, data, _children=(), _op=''):
        self.data = data
        self._prev = set(_children) # Change from tuple to set for efficiency
        self._op = _op
    # So the data print out is more readable
    def __repr__(self):
        return f"Value(data={self.data})"
    # So we can add two Value objects
    def __add__(self, other):
        out = Value(self.data + other.data, (self, other), '+')
        return out
    def __mul__(self, other):
        out = Value(self.data * other.data, (self, other), '*')
        return out


a = Value(2.0)
b = Value(-3.0)
c = Value(10.0)
a*b+c

# (a.__mul__(b)).__add__(c)

: 

In [ ]:
from graphviz import Digraph

def trace(root):
  # builds a set of all nodes and edges in a graph
  nodes, edges = set(), set()
  def build(v):
    if v not in nodes:
      nodes.add(v)
      for child in v._prev:
        edges.add((child, v))
        build(child)
  build(root)
  return nodes, edges

def draw_dot(root):
  dot = Digraph(format='svg', graph_attr={'rankdir': 'LR'}) # LR = left to right
  
  nodes, edges = trace(root)
  for n in nodes:
    uid = str(id(n))
    # for any value in the graph, create a rectangular ('record') node for it
    dot.node(name = uid, label = "{ %s | data %.4f | grad %.4f }" % (n.label, n.data, n.grad), shape='record')
    if n._op:
      # if this value is a result of some operation, create an op node for it
      dot.node(name = uid + n._op, label = n._op)
      # and connect this node to it
      dot.edge(uid + n._op, uid)

  for n1, n2 in edges:
    # connect n1 to the op node of n2
    dot.edge(str(id(n1)), str(id(n2)) + n2._op)

  return dot

: 

In [ ]:
draw_dot(c)

: 

: 